In [3]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


### pip

In [4]:
# !pip install tensorflow-recommenders --user -q
# !pip -q install google-cloud-storage

# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [3]:
!pip freeze | grep tensorflow

tensorflow @ file:///opt/conda/conda-bld/dlenv-tf-2-9-cpu_1656643241567/work/tensorflow-2.9.0rc2-cp37-cp37m-linux_x86_64.whl
tensorflow-cloud==0.1.16
tensorflow-datasets==4.4.0
tensorflow-estimator==2.9.0
tensorflow-hub==0.12.0
tensorflow-io==0.23.1
tensorflow-io-gcs-filesystem==0.26.0
tensorflow-metadata==1.9.0
tensorflow-probability==0.14.1
tensorflow-serving-api==2.9.0
tensorflow-transform==1.9.0


In [2]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [19]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

In [9]:
candidate_features = {
    'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
}

seed_track_features = {
    'pos_seed_track': tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
    'track_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'album_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'track_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'album_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'duration_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'track_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'artist_genres_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'artist_followers_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
}

playlist_context_features = {
    'pid': tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
    'name': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'collaborative': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
    'duration_ms_seed_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'n_songs_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'num_artists_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'num_albums_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=(1)),
    'description_pl': tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
}

playlist_sequence_features = {
    'track_name_pl': tf.io.RaggedFeature(tf.string),
    'artist_name_pl': tf.io.RaggedFeature(tf.string),
    'album_name_pl': tf.io.RaggedFeature(tf.string),
    'track_uri_pl': tf.io.RaggedFeature(tf.string),
    'duration_ms_songs_pl': tf.io.RaggedFeature(tf.float32),
    'artist_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artists_followers_pl': tf.io.RaggedFeature(tf.float32),
    'track_pop_pl': tf.io.RaggedFeature(tf.int64),
    'artist_genres_pl': tf.io.RaggedFeature(tf.string),
}

def parse_playlist_tfrecord_fn(example):
    example = tf.io.parse_single_sequence_example(
        example, 
        sequence_features=playlist_sequence_features, 
        context_features=playlist_context_features
    )
    return example

def parse_seed_track_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=seed_track_features,
    )
    return example

def parse_candidate_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=candidate_features,
    )
    return example

In [12]:
SAMPLE_FILE = "gs://spotify-tfrecords-blog/tfrecords_v1/train/output-00000-of-00796.tfrecord"
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "album_name_can"
    value {
      bytes_list {
        value: "Glee: The Music, The Complete Season Three"
      }
    }
  }
  feature {
    key: "album_name_pl"
    value {
      bytes_list {
        value: "Glee: The Music, The Complete Season One"
        value: "Glee: The Music, The Complete Season Three"
        value: "Glee: The Music, The Complete Season Three"
        value: "Glee: The Music, The Complete Season Three"
        value: "Glee: The Music, The Complete Season Two"
        value: "Glee: The Music, The Complete Season Four"
        value: "Glee: The Music, The Complete Season Two"
        value: "Glee: The Music, The Complete Season Two"
        value: "Glee: The Music, The Complete Season Two"
        value: "Glee: The Music presents Glease"
        value: "Glee: The Music, The Complete Season Four"
        value: "Glee: The Music, The Complete Season Four"
        value: "Glee: The Music, The Complete Season Four"
        value: "Gle

In [14]:
MAX_SEQUENCE_LENGTH = 375

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

# playlists features (parse 1)
parsed_playlist_ds = raw_dataset.map(
    parse_playlist_tfrecord_fn,
    num_parallel_calls=tf.data.AUTOTUNE
).with_options(options)

# function for sequence features
def return_rag_pl_tensors(context, sequence):
        a = sequence['artist_name_pl'].to_tensor(default_value='', shape=[None, 375])
        b = sequence['track_uri_pl'].to_tensor(default_value='', shape=[None, 375])
        c = sequence['track_name_pl'].to_tensor(default_value='', shape=[None, 375])
        d = sequence['duration_ms_songs_pl'].to_tensor(default_value=0.0, shape=[None, 375])
        e = sequence['album_name_pl'].to_tensor(default_value='', shape=[None, 375])
        f = sequence['artist_pop_pl'].to_tensor(default_value=0.0, shape=[None, 375])
        g = sequence['artists_followers_pl'].to_tensor(default_value=0.0, shape=[None, 375])
        h = sequence['track_pop_pl'].to_tensor(default_value=0, shape=[None, 375])
        i = sequence['artist_genres_pl'].to_tensor(default_value='', shape=[None, 375])
        context2 = context.copy()
        context2['artist_name_pl'] = a
        context2['track_uri_pl'] = b
        context2['track_name_pl'] = c
        context2['duration_ms_songs_pl'] = d
        context2['album_name_pl'] = e
        context2['artist_pop_pl'] = f
        context2['artists_followers_pl'] = g
        context2['track_pop_pl'] = h
        context2['artist_genres_pl'] = i
        return context2

# playlists features (parse 2)
parsed_playlist_ds = parsed_playlist_ds.map(
    return_rag_pl_tensors,
    num_parallel_calls=tf.data.AUTOTUNE
).with_options(options)

# seed track features
parsed_seed_ds = raw_dataset.map(
    parse_seed_track_tfrecord_fn,
    num_parallel_calls=tf.data.AUTOTUNE
).with_options(options)

# candidate features
parsed_candidate_ds = raw_dataset.map(
    parse_candidate_tfrecord_fn,
    num_parallel_calls=tf.data.AUTOTUNE
).with_options(options)

In [21]:
for _ in parsed_playlist_ds.batch(3).take(1):
    pprint(_)

{'album_name_pl': <tf.Tensor: shape=(3, 0, 375), dtype=string, numpy=array([], shape=(3, 0, 375), dtype=object)>,
 'artist_genres_pl': <tf.Tensor: shape=(3, 0, 375), dtype=string, numpy=array([], shape=(3, 0, 375), dtype=object)>,
 'artist_name_pl': <tf.Tensor: shape=(3, 0, 375), dtype=string, numpy=array([], shape=(3, 0, 375), dtype=object)>,
 'artist_pop_pl': <tf.Tensor: shape=(3, 0, 375), dtype=float32, numpy=array([], shape=(3, 0, 375), dtype=float32)>,
 'artists_followers_pl': <tf.Tensor: shape=(3, 0, 375), dtype=float32, numpy=array([], shape=(3, 0, 375), dtype=float32)>,
 'collaborative': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'false'],
       [b'false'],
       [b'false']], dtype=object)>,
 'description_pl': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b''],
       [b''],
       [b'']], dtype=object)>,
 'duration_ms_seed_pl': <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[10529275.],
       [ 9145375.],
       [18341276.]], dtype=float32)>,
 